In [1]:
from functools import partial, reduce
from os import rename, makedirs
import os
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime
from typing import Literal

In [3]:
nomenclador = pd.read_excel('../argendata/qa/geonomencladores/geonomenclador.xlsx')
pais = {k:v for k,v in nomenclador[['codigo_fundar', 'desc_fundar']].iloc}
continentes = {k:v for k,v in nomenclador[['codigo_fundar', 'continente_fundar']].iloc}
nivel_agregacion = {k:v for k,v in nomenclador[['codigo_fundar', 'nivel_agregacion']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [4]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

def cast_col(df:pd.DataFrame, col:str, cast_to:Literal['int','float','object'], lista_cambios=lista_cambios):
    old_type = str(df[col].dtype)
    df[col] = df[col].astype(cast_to)
    lista_cambios.append(f"El tipo de dato de la columna {col} fue cambiado {old_type} ->> {cast_to}")
    return df

def cast_colx(col, cast_to):
    return lambda df: cast_col(df=df, col=col, cast_to=cast_to)

SALING_g1.csv

In [5]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre


,year,semestre,ipcf_index
0,2003,2,100.00000
1,2004,1,102.31232
2,2004,2,107.82178
3,2005,1,109.70754
4,2005,2,118.60464


In [6]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,indicador,valor
0,2003,2,ipcf_index,100.00000
1,2004,1,ipcf_index,102.31232
2,2004,2,ipcf_index,107.82178
3,2005,1,ipcf_index,109.70754
4,2005,2,ipcf_index,118.60464


In [7]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre']}

In [8]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g01.csv',
 '../output/SALING1/definitivos/SALING_g01_old.csv')

SALING_g2.csv

In [9]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)

df.head()

country_code year


,country_code,year,ipcf_promedio
0,ARG,2022,695.305760
1,BOL,2021,594.310900
2,BRA,2022,648.757310
3,CHL,2022,865.293987
4,COL,2022,525.825395


In [11]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    nomenclar_paises(code_col='country_code', desc_col=None, new_col=None),
    to_long(primary_keys),
    sort_vals_ascx(primary_keys),
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,country_code,year,indicador,valor
0,ARG,2022,ipcf_promedio,695.305760
1,BOL,2021,ipcf_promedio,594.310900
2,BRA,2022,ipcf_promedio,648.757310
3,CHL,2022,ipcf_promedio,865.293987
4,COL,2022,ipcf_promedio,525.825395


In [12]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year']}

In [13]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g02.csv',
 '../output/SALING1/definitivos/SALING_g02_old.csv')

SALING_g3.csv

In [14]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

country_code year


,country_code,year,ipcf_promedio
0,ARG,1992,699.335543
1,ARG,1993,700.806294
2,ARG,1994,692.341703
3,ARG,1995,657.670162
4,ARG,1996,644.265897


In [16]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
nomenclar_continentes = partial(normalize_countriesx, map=continentes)
nomenclar_nivel_agg = partial(normalize_countriesx, map=nivel_agregacion)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    nomenclar_paises(code_col='country_code', desc_col=None, new_col=None),
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,country_code,year,indicador,valor
0,ARG,1992,ipcf_promedio,699.335543
1,ARG,1993,ipcf_promedio,700.806294
2,ARG,1994,ipcf_promedio,692.341703
3,ARG,1995,ipcf_promedio,657.670162
4,ARG,1996,ipcf_promedio,644.265897


In [17]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year']}

In [18]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g03.csv',
 '../output/SALING1/definitivos/SALING_g03_old.csv')

SALING_g4.csv

In [19]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre region deflactor


,year,semestre,region,deflactor,indice
0,2023,1,Partidos del GBA,ipc,0.940684
1,2023,1,CABA,ipc,1.631530
2,2023,1,Pampeana,ipc,0.997190
3,2023,1,NEA,ipc,0.716571
4,2023,1,NOA,ipc,0.706723


In [20]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    #nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,region,deflactor,indicador,valor
0,2023,1,CABA,ipc,indice,1.631530
1,2023,1,CABA,linea de pobreza,indice,1.602125
2,2023,1,Cuyo,ipc,indice,0.845501
3,2023,1,Cuyo,linea de pobreza,indice,0.866753
4,2023,1,NEA,ipc,indice,0.716571


In [21]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor']}

In [22]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g04.csv',
 '../output/SALING1/definitivos/SALING_g04_old.csv')

SALING_g5.csv

In [23]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre region


,year,semestre,region,indice
0,2003,2,Total,100.000000
1,2003,2,Partidos del GBA,78.890640
2,2003,2,CABA,219.274810
3,2003,2,Pampeana,91.775589
4,2003,2,NEA,68.673203


In [24]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,region,indicador,valor
0,2003,2,CABA,indice,219.274810
1,2003,2,Cuyo,indice,80.942444
2,2003,2,NEA,indice,68.673203
3,2003,2,NOA,indice,77.371071
4,2003,2,Pampeana,indice,91.775589


In [25]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [26]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g05.csv',
 '../output/SALING1/definitivos/SALING_g05_old.csv')

SALING_g6.csv

In [27]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre


,year,semestre,proporcion
0,2003,2,0.834490
1,2004,1,0.869794
2,2004,2,0.839381
3,2005,1,0.852926
4,2005,2,0.860603


In [28]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,indicador,valor
0,2003,2,proporcion,0.834490
1,2004,1,proporcion,0.869794
2,2004,2,proporcion,0.839381
3,2005,1,proporcion,0.852926
4,2005,2,proporcion,0.860603


In [29]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [30]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g06.csv',
 '../output/SALING1/definitivos/SALING_g06_old.csv')

SALING_g7.csv

In [31]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre decil fuente 


,year,semestre,decil,fuente,proporcion
0,2023,1,Decil 1,Ingreso laboral,0.618709
1,2023,1,Decil 2,Ingreso laboral,0.685302
2,2023,1,Decil 3,Ingreso laboral,0.753713
3,2023,1,Decil 4,Ingreso laboral,0.791003
4,2023,1,Decil 5,Ingreso laboral,0.799150


In [33]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['year','semestre', 'decil', 'fuente']),
    sort(['year','semestre', 'decil', 'fuente'])
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,decil,fuente,indicador,valor
0,2023,1,Decil 1,Ingreso de capital,proporcion,0.002274
1,2023,1,Decil 1,Ingreso de jubilaciones,proporcion,0.103480
2,2023,1,Decil 1,Ingreso de transferencias estatales,proporcion,0.243019
3,2023,1,Decil 1,Ingreso laboral,proporcion,0.618709
4,2023,1,Decil 1,Otros ingresos,proporcion,0.032518


In [34]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [35]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g07.csv',
 '../output/SALING1/definitivos/SALING_g07_old.csv')

SALING_g8.csv

In [36]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre edad_jefe fuente 


,year,semestre,edad_jefe,fuente,proporcion
0,2023,1,24 o menos,Ingreso laboral,0.714832
1,2023,1,25 a 34,Ingreso laboral,0.863664
2,2023,1,35 a 44,Ingreso laboral,0.905991
3,2023,1,45 a 54,Ingreso laboral,0.897952
4,2023,1,55 a 64,Ingreso laboral,0.790378


In [38]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['year','semestre','edad_jefe','fuente']),
    sort(['year','semestre','edad_jefe','fuente']),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,edad_jefe,fuente,indicador,valor
0,2023,1,24 o menos,Ingreso de capital,proporcion,0.009679
1,2023,1,24 o menos,Ingreso de jubilaciones,proporcion,0.027072
2,2023,1,24 o menos,Ingreso de transferencias estatales,proporcion,0.072161
3,2023,1,24 o menos,Ingreso laboral,proporcion,0.714832
4,2023,1,24 o menos,Otros ingresos,proporcion,0.176256


In [39]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [40]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g08.csv',
 '../output/SALING1/definitivos/SALING_g08_old.csv')

SALING_g9.csv

In [41]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre genero


,year,semestre,genero,proporcion
0,2003,2,Mujer,0.727663
1,2004,1,Mujer,0.746443
2,2004,2,Mujer,0.719781
3,2005,1,Mujer,0.724348
4,2005,2,Mujer,0.738315


In [43]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,genero,indicador,valor
0,2003,2,Mujer,proporcion,0.727663
1,2003,2,Varon,proporcion,0.856604
2,2004,1,Mujer,proporcion,0.746443
3,2004,1,Varon,proporcion,0.894491
4,2004,2,Mujer,proporcion,0.719781


In [44]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [45]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g09.csv',
 '../output/SALING1/definitivos/SALING_g09_old.csv')

SALING_g10.csv

In [46]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year semestre fuente 


,year,semestre,fuente,indice
0,2003,2,Ingreso de capital,100.000000
1,2003,2,Ingreso de jubilaciones,100.000000
2,2003,2,Ingreso de transferencias estatales,100.000000
3,2003,2,Otros ingresos,100.000000
4,2004,1,Ingreso de capital,48.225368


In [48]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['year','semestre','fuente']),
    sort(['year','semestre','fuente']),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,semestre,fuente,indicador,valor
0,2003,2,Ingreso de capital,indice,100.000000
1,2003,2,Ingreso de jubilaciones,indice,100.000000
2,2003,2,Ingreso de transferencias estatales,indice,100.000000
3,2003,2,Otros ingresos,indice,100.000000
4,2004,1,Ingreso de capital,indice,48.225368


In [49]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [50]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g10.csv',
 '../output/SALING1/definitivos/SALING_g10_old.csv')

SALING_g11.csv

In [51]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano variable


,ano,variable,valor
0,1992,ingresohorrario,100.0
1,1992,ingresomensual,100.0
2,1993,ingresohorrario,100.4
3,1993,ingresomensual,102.5
4,1994,ingresohorrario,113.0


In [53]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    renombrar({'_valor': 'valor'}),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,variable,indicador,valor
0,1992,ingresohorrario,valor,100.0
1,1992,ingresomensual,valor,100.0
2,1993,ingresohorrario,valor,100.4
3,1993,ingresomensual,valor,102.5
4,1994,ingresohorrario,valor,113.0


In [54]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [55]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g11.csv',
 '../output/SALING1/definitivos/SALING_g11_old.csv')

SALING_g12.csv

In [56]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 12

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

mes ano


,mes,ano,indice
0,1,2016,NaN
1,2,2016,NaN
2,3,2016,NaN
3,4,2016,NaN
4,5,2016,NaN


In [61]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    renombrar({'mes ': 'mes'}),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,mes,ano,indicador,valor
0,1,2016,indice,NaN
1,1,2017,indice,100.0
2,1,2018,indice,101.0
3,1,2019,indice,90.0
4,1,2020,indice,85.0


In [62]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [63]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g12.csv',
 '../output/SALING1/definitivos/SALING_g12_old.csv')

SALING_g13.csv

In [64]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 13

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano


,ano,salario
0,1947,7834.16
1,1948,9185.16
2,1949,9151.66
3,1950,8596.90
4,1951,8125.48


In [65]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,indicador,valor
0,1947,salario,7834.16
1,1948,salario,9185.16
2,1949,salario,9151.66
3,1950,salario,8596.90
4,1951,salario,8125.48


In [66]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [67]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g13.csv',
 '../output/SALING1/definitivos/SALING_g13_old.csv')

SALING_g14.csv

In [68]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 14

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

pais


,pais,salariohorario
0,HND,2.7
1,GTM,3.3
2,PER,3.4
3,SLV,3.6
4,MEX,3.7


In [69]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='pais', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,pais,indicador,valor
0,ARG,salariohorario,5.3
1,BOL,salariohorario,4.8
2,BRA,salariohorario,5.2
3,CHL,salariohorario,7.5
4,COL,salariohorario,3.7


In [70]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [71]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g14.csv',
 '../output/SALING1/definitivos/SALING_g14_old.csv')

SALING_g15.csv

In [72]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 15

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano variable


,ano,variable,valor
0,1992,Argentina,6.2
1,1992,AmericaLatina,3.7
2,1993,Argentina,6.2
3,1993,AmericaLatina,3.9
4,1994,Argentina,7.0


In [75]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    replace(col='variable', curr_value='AmericaLatina', new_value='América Latina'),
    to_long(primary_keys, value_name="_valor"),
    renombrar({'_valor': 'valor'}),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,variable,indicador,valor
0,1992,América Latina,valor,3.7
1,1992,Argentina,valor,6.2
2,1993,América Latina,valor,3.9
3,1993,Argentina,valor,6.2
4,1994,América Latina,valor,4.1


In [76]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [77]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g15.csv',
 '../output/SALING1/definitivos/SALING_g15_old.csv')

SALING_g16.csv

In [78]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 16

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano variable


,ano,variable,valor
0,1992,Empresarios,140
1,1992,Asalariadosfirmasgrandes,86
2,1992,Asalariadospúblico,85
3,1992,Profesionales,175
4,1992,Asalariadospequeñas,68


In [81]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    replace(col='variable', curr_value='Asalariadosfirmasgrandes', new_value='Asalariados firmas grandes'),
    replace(col='variable', curr_value='Asalariadospúblico', new_value='Asalariados sector público'),
    replace(col='variable', curr_value='Asalariadospequeñas', new_value='Asalariados pequeñas empresas'),
    to_long(primary_keys, value_name="_valor"),
    renombrar({'_valor': 'valor'}),
    sort(primary_keys),
    # 
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,variable,indicador,valor
0,1992,Asalariados firmas grandes,valor,86
1,1992,Asalariados pequeñas empresas,valor,68
2,1992,Asalariados sector público,valor,85
3,1992,Cuentapropistas,valor,87
4,1992,Empresarios,valor,140


In [82]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [83]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g16.csv',
 '../output/SALING1/definitivos/SALING_g16_old.csv')

SALING_g17.csv

In [84]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 17

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano mes variable


,ano,mes,variable,valor
0,2016,1,privadonoregistrado,NaN
1,2016,1,privadoregistrado,1000.0
2,2016,1,sectorpublico,990.0
3,2016,2,privadonoregistrado,NaN
4,2016,2,privadoregistrado,990.0


In [86]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    replace(col='variable', curr_value='privadonoregistrado', new_value='privado no registrado'),
    replace(col='variable', curr_value='privadoregistrado', new_value='privado registrado'),
    replace(col='variable', curr_value='sectorpublico', new_value='sector publico'),
    to_long(primary_keys, value_name="_valor"),
    renombrar({'_valor': 'valor'}),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,mes,variable,indicador,valor
0,2016,1,privado no registrado,valor,NaN
1,2016,1,privado registrado,valor,1000.0
2,2016,1,sector publico,valor,990.0
3,2016,2,privado no registrado,valor,NaN
4,2016,2,privado registrado,valor,990.0


In [87]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [88]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g17.csv',
 '../output/SALING1/definitivos/SALING_g17_old.csv')

SALING_g18.csv

In [89]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 18

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

pais variable


,pais,variable,valor
0,Panamá,Empleadores,3938.8
1,Panamá,Asalariadosgrandes,1903.3
2,Panamá,publico,2967.6
3,Panamá,profesionales,1611.7
4,Panamá,Asalariadosmicro,741.7


In [91]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    replace(col='variable', curr_value='Asalariadosgrandes', new_value='Asalariados grandes'),
    replace(col='variable', curr_value='Asalariadosmicro', new_value='Asalariados micro'),
    to_long(primary_keys, value_name="_valor"),
    renombrar({'_valor': 'valor'}),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,pais,variable,indicador,valor
0,Argentina,Asalariados grandes,valor,1200.5
1,Argentina,Asalariados micro,valor,594.3
2,Argentina,Cuentapropia,valor,601.9
3,Argentina,Empleadores,valor,1344.4
4,Argentina,Media,valor,978.7


In [92]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [93]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g18.csv',
 '../output/SALING1/definitivos/SALING_g18_old.csv')

SALING_g19.csv

In [94]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 19

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

region variable


,region,variable,valor
0,GBA,Total,103.5
1,GBA,Baja,67.7
2,GBA,Media,84.0
3,GBA,Alta,150.8
4,Pampeana,Total,94.6


In [95]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,region,variable,indicador,valor
0,Cuyo,Alta,valor,113.5
1,Cuyo,Baja,valor,54.5
2,Cuyo,Media,valor,71.5
3,Cuyo,Total,valor,83.0
4,GBA,Alta,valor,150.8


In [96]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [97]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g19.csv',
 '../output/SALING1/definitivos/SALING_g19_old.csv')

SALING_g20.csv

In [98]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 20

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano_trim variable


,ano_trim,variable,valor
0,2016-I,Total,98.81
1,2016-I,GBA,102.52
2,2016-I,Pampeana,91.71
3,2016-I,Cuyo,81.85
4,2016-I,NOA,85.74


In [99]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys,value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano_trim,variable,indicador,valor
0,2016-I,Cuyo,valor,81.85
1,2016-I,GBA,valor,102.52
2,2016-I,NEA,valor,83.42
3,2016-I,NOA,valor,85.74
4,2016-I,Pampeana,valor,91.71


In [100]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [101]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g20.csv',
 '../output/SALING1/definitivos/SALING_g20_old.csv')

SALING_g21.csv

In [102]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 21

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

edad variable


,edad,variable,valor
0,25,hombres1,117726
1,25,mujeres1,86533
2,25,hombres2,111610
3,25,mujeres2,93361
4,26,hombres1,110946


In [105]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,edad,variable,indicador,valor
0,25,hombres1,valor,117726
1,25,hombres2,valor,111610
2,25,mujeres1,valor,86533
3,25,mujeres2,valor,93361
4,26,hombres1,valor,110946


In [106]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [107]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g21.csv',
 '../output/SALING1/definitivos/SALING_g21_old.csv')

SALING_g22.csv

In [108]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 22

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

edad variable


,edad,variable,valor
0,25,Hombres1,740
1,25,Mujeres1,687
2,25,Hombres2,715
3,25,Mujeres2,738
4,26,Hombres1,693


In [109]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,edad,variable,indicador,valor
0,25,Hombres1,valor,740
1,25,Hombres2,valor,715
2,25,Mujeres1,valor,687
3,25,Mujeres2,valor,738
4,26,Hombres1,valor,693


In [110]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [111]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g22.csv',
 '../output/SALING1/definitivos/SALING_g22_old.csv')

SALING_g23.csv

In [112]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 23

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

ano variable


,ano,variable,valor
0,1992,Mujeres,11178
1,1992,Hombres,15652
2,1992,(15-24),9325
3,1992,(25-64),15414
4,1992,(65-),10078


In [113]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,ano,variable,indicador,valor
0,1992,(15-24),valor,9325
1,1992,(25-64),valor,15414
2,1992,(65-),valor,10078
3,1992,Hombres,valor,15652
4,1992,Mujeres,valor,11178


In [114]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [115]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g23.csv',
 '../output/SALING1/definitivos/SALING_g23_old.csv')

SALING_g24.csv

In [116]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 24

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,salario_real_base1970
0,1935,51
1,1936,50
2,1937,51
3,1938,53
4,1939,52


In [117]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1935,salario_real_base1970,51
1,1936,salario_real_base1970,50
2,1937,salario_real_base1970,51
3,1938,salario_real_base1970,53
4,1939,salario_real_base1970,52


In [118]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [119]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g24.csv',
 '../output/SALING1/definitivos/SALING_g24_old.csv')

SALING_g25.csv

In [120]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 25

archivo = f'{SUBTOP}_g{str(grafico_n).zfill(2)}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,salario_real_ppa_consumo_privado_2017
0,USA,1929,1881.734056
1,USA,1929,100.000000
2,USA,1930,1880.605043
3,USA,1930,100.000000
4,USA,1931,1908.079312


In [121]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,indicador,valor
0,ARG,1935,salario_real_ppa_consumo_privado_2017,1074.569920
1,ARG,1935,salario_real_ppa_consumo_privado_2017,58.772606
2,ARG,1936,salario_real_ppa_consumo_privado_2017,1052.470928
3,ARG,1936,salario_real_ppa_consumo_privado_2017,54.937676
4,ARG,1937,salario_real_ppa_consumo_privado_2017,1066.366227


In [122]:
diccionario_cambios

{'SALING_g01': ['Long estricto: [year, semestre, ipcf_index] -> [year, semestre, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre'],
 'SALING_g02': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g03': ['Usando los valores de la columna country_code, se verificó que todos los códigos se encuentran en el nomenclador vigente al 2024-04-30',
  'Long estricto: [country_code, year, ipcf_promedio] -> [country_code, year, valor, indicador]',
  'Ordené las filas por las columnas: country_code, year'],
 'SALING_g04': ['Long estricto: [year, semestre, region, deflactor, indice] -> [year, semestre, region, deflactor, valor, indicador]',
  'Ordené las filas por las columnas: year, semestre, region, deflactor'],
 'SALING_

In [123]:
exportar(_df)

('../output/SALING1/definitivos/SALING_g25.csv',
 '../output/SALING1/definitivos/SALING_g25_old.csv')

In [124]:
SUBTOP = 'SALING'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

Files matching the pattern 're.compile('.*old.*')' moved to '../output/SALING1/old'.


In [125]:
import json
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

with open(f'../output/SALING1/definitivos/paso_a_definitivos_{today}.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
